In [1]:
import pandas as pd
import numpy as np
import json
from collections import namedtuple

In [2]:
df = pd.read_csv('../data/one_travel_chain.csv')
# 提取所有不同的 grid IDs
unique_grid_ids = pd.unique(df[['grid_id_o', 'grid_id_d']].values.ravel('K'))

# 使用 factorize 方法为所有不同的 grid IDs 分配新的编码
new_codes, unique = pd.factorize(unique_grid_ids, sort=True)

# 创建一个字典来映射旧的 grid IDs 到新的编码
grid_id_mapping = dict(zip(unique, new_codes))

df['grid_new_o'] = df['grid_id_o'].map(grid_id_mapping)
df['grid_new_d'] = df['grid_id_d'].map(grid_id_mapping)
df.head()

,who,date,seiqd,mode,lambda_o,phi_o,lambda_d,phi_d,migrt,weekend,...,LU_Residence,LU_Science_&_Education,LU_Special,LU_Transportation,LU_Wild,trip_routelen,grid_id_o,grid_id_d,grid_new_o,grid_new_d
0,77678983,20190101,1,1,114.101082,22.577668,114.101268,22.556263,20190701,False,...,0.945229,0.319338,0.122423,0.000000,0.000000,2917.227,20850.0,17637,15,152
1,77678983,20190101,2,1,114.101268,22.556263,114.106437,22.591168,20190701,False,...,1.657962,0.000000,0.000000,0.088206,0.055425,4326.275,17637.0,22994,152,112
2,77678983,20190102,1,1,114.101082,22.577668,113.928502,22.521493,20190701,False,...,0.997163,0.000000,0.000000,0.000000,0.000000,21444.991,20850.0,11854,15,120
3,77678983,20190102,2,1,113.928502,22.521493,114.101082,22.577668,20190701,False,...,0.063617,0.213769,0.000000,0.000000,0.233169,21444.991,11854.0,20850,120,15
4,77678983,20190103,1,1,113.960863,22.545302,113.928502,22.521493,20190701,False,...,0.997163,0.000000,0.000000,0.000000,0.000000,5038.988,15794.0,11854,129,120


In [3]:
def build_chain(group):
    chains = []
    current_chain = [group['grid_id_o'].iloc[0]]
    
    for i in range(len(group) - 1):
        current_chain.append(group['grid_id_d'].iloc[i])
        # Check if the next 'o' is different from the current 'd'
        if group['grid_id_d'].iloc[i] != group['grid_id_o'].iloc[i + 1]:
            chains.append(current_chain)
            current_chain = [group['grid_id_o'].iloc[i + 1]]
    
    # Append the last destination and the final chain
    current_chain.append(group['grid_id_d'].iloc[-1])
    chains.append(current_chain)
    
    return chains

def build_id_chain(group):
    chains = []
    current_chain = [group['grid_new_o'].iloc[0]]
    
    for i in range(len(group) - 1):
        current_chain.append(group['grid_new_d'].iloc[i])
        # Check if the next 'o' is different from the current 'd'
        if group['grid_new_d'].iloc[i] != group['grid_new_o'].iloc[i + 1]:
            chains.append(current_chain)
            current_chain = [group['grid_new_o'].iloc[i + 1]]
    
    # Append the last destination and the final chain
    current_chain.append(group['grid_new_d'].iloc[-1])
    chains.append(current_chain)
    
    return chains

def build_both_chains(group):
    travel_chain = build_chain(group)
    id_chain = build_id_chain(group)
    return pd.Series({'travel_chain': travel_chain, 'id_chain': id_chain})

In [4]:
# Group by 'date' and apply the chain-building function
grouped1 = df[df['date']<df['migrt']]
grouped2 = df[df['date']>df['migrt']]
grouped1 = grouped1.groupby('date').apply(build_both_chains).reset_index()
grouped2 = grouped2.groupby('date').apply(build_both_chains).reset_index()

In [5]:
# Define the namedtuple type
TravelData = namedtuple('TravelChain', ['date', 'travel_chain','id_chain'])

In [6]:
# Convert each row to a namedtuple
namedtuples_list1 = [TravelData(row.date, row.travel_chain[0], row.id_chain[0]) for _, row in grouped1.iterrows()]
namedtuples_list2 = [TravelData(row.date, row.travel_chain[0], row.id_chain[0]) for _, row in grouped2.iterrows()]
namedtuples_all = namedtuples_list1+namedtuples_list2

In [7]:
def int64_converter(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    raise TypeError

dicts_list1 = [nt._asdict() for nt in namedtuples_list1]
with open('../data/before_migrt.json', 'w') as file:
    json.dump(dicts_list1, file, indent=4,default=int64_converter)

dicts_list2 = [nt._asdict() for nt in namedtuples_list2]
with open('../data/after_migrt.json', 'w') as file:
    json.dump(dicts_list2, file, indent=4,default=int64_converter)

dicts_list_all = [nt._asdict() for nt in namedtuples_all]
with open('../data/all_traj.json', 'w') as file:
    json.dump(dicts_list_all, file, indent=4,default=int64_converter)

In [8]:
with open('../data/all_traj.json', 'r') as file:
    loaded_dicts_all = json.load(file)
loaded_namedtuples_all = [TravelData(**d) for d in loaded_dicts_all]

with open('../data/before_migrt.json', 'r') as file:
    loaded_dicts_all = json.load(file)
loaded_namedtuples_before = [TravelData(**d) for d in loaded_dicts_all]

In [9]:
loaded_namedtuples_all

[TravelChain(date=20190101, travel_chain=[20850.0, 17637, 22994], id_chain=[15, 152, 112]),
 TravelChain(date=20190102, travel_chain=[20850.0, 11854, 20850], id_chain=[15, 120, 15]),
 TravelChain(date=20190103, travel_chain=[15794.0, 11854, 16557, 22994], id_chain=[129, 120, 169, 112]),
 TravelChain(date=20190104, travel_chain=[22994.0, 11854, 15483, 20850], id_chain=[112, 120, 39, 15]),
 TravelChain(date=20190106, travel_chain=[22994.0, 15103, 16201], id_chain=[112, 87, 88]),
 TravelChain(date=20190107, travel_chain=[20850.0, 11854, 12966, 11854, 10472], id_chain=[15, 120, 27, 120, 56]),
 TravelChain(date=20190108, travel_chain=[18359.0, 11854, 18359, 20850], id_chain=[122, 120, 122, 15]),
 TravelChain(date=20190109, travel_chain=[22994.0, 11854, 16492, 11854, 14719, 18359], id_chain=[112, 120, 126, 120, 7, 122]),
 TravelChain(date=20190110, travel_chain=[20850.0, 11854, 20850], id_chain=[15, 120, 15]),
 TravelChain(date=20190111, travel_chain=[20850.0, 11854, 33669, 22994], id_chain=